In [1]:
from __future__ import print_function, division
import torch
import matplotlib.pyplot as plt
import argparse,os
import pandas as pd
import cv2
import numpy as np
import random
import math
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from models.CDCNs import Conv2d_cd, CDCN, CDCNpp
from Load_OULUNPU_train import Spoofing_train, Normaliztion, ToTensor, RandomHorizontalFlip, Cutout, RandomErasing
from Load_OULUNPU_valtest import Spoofing_valtest, Normaliztion_valtest, ToTensor_valtest

import torch.nn.functional as F
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import copy
import pdb
from utils import AvgrageMeter, accuracy, performances

In [2]:
train_image_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Train_files/'            
val_image_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Dev_files/'   
test_image_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Test_files/'   
# 文件夹内包含原来的视频.avi文件和其中每一帧的图像1_1_01_1_frame92.jpg
# 以及每一帧的ROi region of interest在.txt文件中 文件内容类似0,455,779,638,756

map_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Train_depth/'   
val_map_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Dev_depth/'   
test_map_dir = '/mnt/hdd.user/datasets/Oulu-NPU/Test_depth/' 
# 文件名1_2_11_3_frame140_depth.jpg

train_list = '/mnt/hdd.user/datasets/Oulu-NPU/Protocols/Protocol_1/Train.txt'
val_list = '/mnt/hdd.user/datasets/Oulu-NPU/Protocols/Protocol_1/Dev.txt'
test_list =  '/mnt/hdd.user/datasets/Oulu-NPU/Protocols/Protocol_1/Test.txt'

device = torch.device("cuda:8")

In [3]:
def contrast_depth_conv(input):
    ''' compute contrast depth in both of (out, label) '''
    '''
        input  32x32
        output 8x32x32
    '''

    # [8, 3, 3]
    kernel_filter_list =[
                        [[1,0,0],[0,-1,0],[0,0,0]], [[0,1,0],[0,-1,0],[0,0,0]], [[0,0,1],[0,-1,0],[0,0,0]],
                        [[0,0,0],[1,-1,0],[0,0,0]], [[0,0,0],[0,-1,1],[0,0,0]],
                        [[0,0,0],[0,-1,0],[1,0,0]], [[0,0,0],[0,-1,0],[0,1,0]], [[0,0,0],[0,-1,0],[0,0,1]]
                        ]
    
    kernel_filter = np.array(kernel_filter_list, np.float32)

    kernel_filter = torch.from_numpy(kernel_filter.astype(np.float)).float().to(device)
    # weights (in_channel, out_channel, kernel, kernel)
    kernel_filter = kernel_filter.unsqueeze(dim=1) # [1, 8, 3, 3]
    # [32, 32]--[1, 32, 32]--[8,32,32](进行完全拷贝的维度扩展)
    input = input.unsqueeze(dim=1).expand(input.shape[0], 8, input.shape[1],input.shape[2])
    # 输出[8,32,32]，group=8相当于每个channel分别和对应的channel的kernel进行卷积运算，得到输出对应的channel
    contrast_depth = F.conv2d(input, weight=kernel_filter, groups=8)  # depthwise conv

    return contrast_depth


class Contrast_depth_loss(nn.Module):    # Pearson range [-1, 1] so if < 0, abs|loss| ; if >0, 1- loss
    def __init__(self):
        super(Contrast_depth_loss,self).__init__()
        return
    def forward(self, out, label): 
        '''
        compute contrast depth in both of (out, label),
        then get the loss of them
        tf.atrous_convd match tf-versions: 1.4
        '''
        # 根据out和label分别构造depth_conv，之后计算mseloss
        contrast_out = contrast_depth_conv(out)
        contrast_label = contrast_depth_conv(label)
        
        
        criterion_MSE = nn.MSELoss().to(device)
    
        loss = criterion_MSE(contrast_out, contrast_label)
        #loss = torch.pow(contrast_out - contrast_label, 2)
        #loss = torch.mean(loss)
    
        return loss
    
parser = argparse.ArgumentParser()
parser.add_argument('--gpu', type=int, default=1, help='the gpu id used for predict')
parser.add_argument('--lr', type=float, default=0.0001, help='initial learning rate')  
parser.add_argument('--batchsize', type=int, default=7, help='initial batchsize')  
parser.add_argument('--step_size', type=int, default=500, help='how many epochs lr decays once')  # 500 
parser.add_argument('--gamma', type=float, default=0.5, help='gamma of optim.lr_scheduler.StepLR, decay of lr')
parser.add_argument('--echo_batches', type=int, default=50, help='how many batches display once')  # 50
parser.add_argument('--epochs', type=int, default=1400, help='total training epochs')
parser.add_argument('--log', type=str, default="CDCNpp_test", help='log and save model name')
# parser.add_argument('--finetune', action='store_true', default=False, help='whether finetune other models')

args = parser.parse_args(args=[])
print(args)

isExists = os.path.exists(args.log)
if not isExists:
    os.makedirs(args.log)
log_file = open(args.log+'/'+ args.log+'_log_P1.txt', 'w')

Namespace(batchsize=7, echo_batches=50, epochs=1400, gamma=0.5, gpu=1, log='CDCNpp_test', lr=0.0001, step_size=500)


In [4]:
echo_batches = args.echo_batches
log_file.write('Oulu-NPU, P1:\n ')
log_file.flush()

model = CDCNpp(basic_conv=Conv2d_cd, theta=0.7).to(device)
model.load_state_dict(torch.load("./CDCNpp_P1/CDCNpp_P1_299.pkl"))

lr = args.lr
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.00005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=args.step_size, gamma=args.gamma)
criterion_absolute_loss = nn.MSELoss().to(device)
criterion_contrastive_loss = Contrast_depth_loss().to(device)

In [5]:
#     val_threshold, test_threshold, val_ACC, val_ACER, test_ACC, test_APCER, test_BPCER, test_ACER, test_ACER_test_threshold = performances(map_score_val_filename, map_score_test_filename)

In [6]:
# map_score_list

In [7]:
model.eval()

with torch.no_grad():
    ###########################################
    '''                val             '''
    ###########################################
    # val for threshold
    val_data = Spoofing_valtest(val_list, val_image_dir, val_map_dir, transform=transforms.Compose([Normaliztion_valtest(), ToTensor_valtest()]))
    dataloader_val = DataLoader(val_data, batch_size=1, shuffle=False, num_workers=1)
    
    
    map_score_list = []
    cnt = 0
    for i, sample_batched in enumerate(dataloader_val):
        inputs, spoof_label = sample_batched['image_x'].to(device), sample_batched['spoofing_label'].to(device)
        val_maps = sample_batched['val_map_x'].to(device)   # binary map from PRNet
        
        map_score = 0.0
        for frame_t in range(inputs.shape[1]):
            map_x, embedding, x_Block1, x_Block2, x_Block3, x_input =  model(inputs[:,frame_t,:,:,:])
            
            score_norm = torch.sum(map_x)/torch.sum(val_maps[:,frame_t,:,:])
            map_score += score_norm
        map_score = map_score/inputs.shape[1]
        print(f"{i}:{map_score}")
        cnt += 1
        break
        if map_score == np.inf or map_score == np.nan:
            break
        if cnt > 5:
            break

/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/opt/conda/lib/python3.8/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(


0:0.0014561937423422933


In [8]:
val_data[0]['image_x'].shape

torch.Size([8, 3, 256, 256])

In [9]:
print(inputs.shape)
print(val_maps.shape)
print(map_x.shape)

torch.Size([1, 8, 3, 256, 256])
torch.Size([1, 8, 32, 32])
torch.Size([1, 32, 32])


In [10]:
for frame_t in range(inputs.shape[1]):
    map_x, embedding, x_Block1, x_Block2, x_Block3, x_input =  model(inputs[:,frame_t,:,:,:])
    print(torch.sum(map_x))
    print(torch.sum(val_maps[:,frame_t,:,:]))
    score_norm = torch.sum(map_x)/torch.sum(val_maps[:,frame_t,:,:])
    print(score_norm)
    map_score += score_norm
map_score = map_score/inputs.shape[1]

tensor(68.2005, device='cuda:8', grad_fn=<SumBackward0>)
tensor(38637., device='cuda:8')
tensor(0.0018, device='cuda:8', grad_fn=<DivBackward0>)
tensor(48.8168, device='cuda:8', grad_fn=<SumBackward0>)
tensor(38888., device='cuda:8')
tensor(0.0013, device='cuda:8', grad_fn=<DivBackward0>)
tensor(65.8934, device='cuda:8', grad_fn=<SumBackward0>)
tensor(38477., device='cuda:8')
tensor(0.0017, device='cuda:8', grad_fn=<DivBackward0>)
tensor(49.8434, device='cuda:8', grad_fn=<SumBackward0>)
tensor(39888., device='cuda:8')
tensor(0.0012, device='cuda:8', grad_fn=<DivBackward0>)
tensor(81.2453, device='cuda:8', grad_fn=<SumBackward0>)
tensor(41691., device='cuda:8')
tensor(0.0019, device='cuda:8', grad_fn=<DivBackward0>)
tensor(44.3709, device='cuda:8', grad_fn=<SumBackward0>)
tensor(41917., device='cuda:8')
tensor(0.0011, device='cuda:8', grad_fn=<DivBackward0>)
tensor(69.3602, device='cuda:8', grad_fn=<SumBackward0>)
tensor(42156., device='cuda:8')
tensor(0.0016, device='cuda:8', grad_fn=<

In [11]:
val_maps[:,2]

tensor([[[  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         [  0.,   0.,   0.,  ...,   0.,   0.,   0.],
         ...,
         [187., 185., 184.,  ...,   0.,   0.,   0.],
         [182., 180., 176.,  ...,   0.,   0.,   0.],
         [180., 179., 177.,  ...,   0.,   0.,   0.]]], device='cuda:8')

In [16]:
train_data = Spoofing_train(train_list, train_image_dir, map_dir, transform=transforms.Compose([RandomErasing(), RandomHorizontalFlip(),  ToTensor(), Cutout(), Normaliztion()]))
dataloader_train = DataLoader(train_data, batch_size=args.batchsize, shuffle=True, num_workers=1)
for i, sample_batched in enumerate(dataloader_train):
    sample_batch = train_data[0]
    inputs, map_label, spoof_label = sample_batched['image_x'].to(device), sample_batched['map_x'].to(device), sample_batched['spoofing_label'].to(device) 
    map_x, embedding, x_Block1, x_Block2, x_Block3, x_input =  model(inputs)
    break

print(inputs.shape)
print(map_label.shape)
print(map_x.shape)

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/workspace/CDCN-master/CVPR2020_paper_codes/Load_OULUNPU_train.py", line 227, in __getitem__
    sample = self.transform(sample)
  File "/opt/conda/lib/python3.8/site-packages/torchvision/transforms/transforms.py", line 60, in __call__
    img = t(img)
  File "/workspace/CDCN-master/CVPR2020_paper_codes/Load_OULUNPU_valtest.py", line 57, in __call__
    image_x, val_map_x, spoofing_label = sample['image_x'],sam

In [14]:
train_data[0]['image_x'].shape

torch.Size([3, 256, 256])